# **🟠최근 1000건 기사 크롤링 및 키워드 카운트🟠**

In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json

### 뉴스 데이터 가져오기

In [ ]:
client_id = 'qzIQMEBIvbTVu7zhgl2m'
client_secret = 'rgOrkg84Iu'

In [ ]:
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

In [ ]:
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url) #[CODE 1]

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

In [ ]:
def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt':cnt, 'title':title, 'description': description, 'org_link':org_link, 'link': org_link, 'pDate':pDate})
    return

In [ ]:
node = 'news' #크롤링할 대상
srcText = input('검색어를 입력하세요: ')
cnt = 0
jsonResult = []
jsonResponse = getNaverSearch(node, srcText, 1, 100) #[CODE 2]
total = jsonResponse['total']

while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
    for post in jsonResponse['items']:
        cnt += 1
        getPostData(post, jsonResult, cnt) #[CODE 3]
    
    start = jsonResponse['start'] + jsonResponse['display']
    jsonResponse = getNaverSearch(node, srcText, start, 100) #[CODE 2]

print('전체 검색 : %d 건' %total)

with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
    jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
    outfile.write(jsonFile)

print("가져온 데이터 : %d 건" %(cnt))
print('%s_naver_%s.json SAVED' % (srcText, node))

검색어를 입력하세요: 카카오페이
[2022-04-04 09:49:49.134759] Url Request Success
[2022-04-04 09:49:50.544202] Url Request Success
[2022-04-04 09:49:51.771193] Url Request Success
[2022-04-04 09:49:53.014256] Url Request Success
[2022-04-04 09:49:54.248224] Url Request Success
[2022-04-04 09:49:55.411722] Url Request Success
[2022-04-04 09:49:56.920950] Url Request Success
[2022-04-04 09:49:58.242723] Url Request Success
[2022-04-04 09:49:59.519725] Url Request Success
[2022-04-04 09:50:00.780734] Url Request Success
HTTP Error 400: Bad Request
[2022-04-04 09:50:01.713772] Error for URL : https://openapi.naver.com/v1/search/news.json?query=%EC%B9%B4%EC%B9%B4%EC%98%A4%ED%8E%98%EC%9D%B4&start=1001&display=100
전체 검색 : 120025 건
가져온 데이터 : 1000 건
카카오페이_naver_news.json SAVED


### 가져온 데이터를 데이터 프레임화 하기

In [ ]:
import pandas as pd
import re

In [ ]:
def clean_html(x):
  #https://predictor-ver1.tistory.com/4
  x = re.sub("\&\w*\;","",x)
  x = re.sub("<.*?>","",x)
  return x

In [ ]:
df_json = pd.DataFrame(jsonResult)
df_json

,cnt,title,description,org_link,link,pDate
0,1,"[단독] 토스 이승건 대표, 테슬라 자동차 20대 증여...“팀원들 고마워”","20대의 차량은 토스, 토스뱅크, 토스증권, 토스<b>페이</b>먼츠 등 각 계열사...",https://www.dailian.co.kr/news/view/1099901/?s...,https://www.dailian.co.kr/news/view/1099901/?s...,2022-04-04 18:42:00
1,2,"송재봉 청주시장 예비후보, 보편적 재난지원금 지급 공약","이어 청주<b>페이</b> 발행 총량 및 할인율 확대, 코로나 블루 극복 전담TF ...",https://www.nocutnews.co.kr/news/5734331,https://www.nocutnews.co.kr/news/5734331,2022-04-04 18:36:00
2,3,‘택진이형’도 못 피해갔다…재계 덮친 ‘연봉 0원’ 논란,한 주주가 카카오·<b>카카오페이</b> 대표의 최저임금 선언을 언급하며 “기우성 ...,https://www.joongang.co.kr/article/25060816,https://www.joongang.co.kr/article/25060816,2022-04-04 18:17:00
3,4,"아시아나, 25개월 만에 하와이 재운항…티웨이항공, A330 비즈니스 업그레이드...",<b>카카오페이</b>로 10만 원 이상 결제할 경우 3000원 할인 쿠폰도 주어진...,http://www.sisaon.co.kr/news/articleView.html?...,http://www.sisaon.co.kr/news/articleView.html?...,2022-04-04 18:02:00
4,5,[2022년 1분기 인기 체크카드] 1위 ‘우리 010PAY 체크카드’,4위는 ‘네이버<b>페이</b> 우리카드 체크’다. 연회비 5000원으로 전월실적 ...,http://www.fntimes.com/html/view.php?ud=202204...,http://www.fntimes.com/html/view.php?ud=202204...,2022-04-04 17:56:00
...,...,...,...,...,...,...
995,996,하루에 6000억… 'OO<b>페이</b>' 결제 역대 최대,한국은행이 23일 공개한 '전자지급 서비스 이용 현황'에 따르면 지난해 <b>카카오...,https://www.etoday.co.kr/news/view/2116490,https://www.etoday.co.kr/news/view/2116490,2022-03-23 12:02:00
996,997,'○○<b>페이</b>' 선불전자지급 42% 급증…작년 하루 이용액 6천억원대,비대면 거래 확대로 전자지급결제대행 서비스 이용액도 28%↑ 선볼전자지급·전자지급결...,http://yna.kr/AKR20220323067900002?did=1195m,http://yna.kr/AKR20220323067900002?did=1195m,2022-03-23 12:01:00
997,998,"코로나19 영향, 지난해 전자지급결제 이용금액 ‘사상최대’","<b>카카오페이</b>, 토스 등에서 미리 선불금을 충전해 물건을 사거나 교통요금을...",https://news.kbs.co.kr/news/view.do?ncd=542202...,https://news.kbs.co.kr/news/view.do?ncd=542202...,2022-03-23 12:01:00
998,999,코로나에 비대면 거래 급증…간편결제 6000억 돌파,삼성페이·<b>카카오페이</b> 등으로 대변되는 간편결제 이용금액도 6000억원을 ...,https://biz.chosun.com/policy/policy_sub/2022/...,https://biz.chosun.com/policy/policy_sub/2022/...,2022-03-23 12:01:00


In [ ]:
df_json['title'] = df_json['title'].apply(lambda x: clean_html(x))
df_json['description'] = df_json['description'].apply(lambda x: clean_html(x))
df_json[350:400]

,cnt,title,description,org_link,link,pDate
350,351,박세진 투게더펀딩 신임 대표 “초일류 테크 금융사로 이끌 것”,"투게더앱스는 최근 핀다, 핀크, 뱅크샐러드, 카카오페이 등과 제휴를 맺고 영업채널을...",http://www.fntimes.com/html/view.php?ud=202203...,http://www.fntimes.com/html/view.php?ud=202203...,2022-03-30 10:56:00
351,352,"카카오페이증권, 해외 주식 소수점 거래 시작",뉴데일리 홍승빈 기자 = 카카오페이증권이 실시간 수준으로 1000원부터 투자할 수 ...,https://biz.newdaily.co.kr/site/data/html/2022...,https://biz.newdaily.co.kr/site/data/html/2022...,2022-03-30 10:56:00
352,353,"카카오, 조직 개편 마무리… 새로운 리더십 시대 연다",새로운 리더십 출범과 기존 조직 개편까지 마무리 지은 카카오는 기존 사업들을 더욱 ...,http://www.sisaweek.com/news/articleView.html?...,http://www.sisaweek.com/news/articleView.html?...,2022-03-30 10:54:00
353,354,[소액주주 수난시대] 국내 주식시장 이탈 '가속',카카오페이 류영준 전 대표 등 임원 8명은 카카오페이 상장 후 한 달 만에 스톡옵션...,https://www.ebn.co.kr/news/view/1525133/?sc=Naver,https://www.ebn.co.kr/news/view/1525133/?sc=Naver,2022-03-30 10:54:00
354,355,"카카오페이증권, 해외 주식 소수점 거래 시작",카카오페이증권은 실시간 수준으로 1000원부터 투자할 수 있는 해외 주식 소수점 거...,http://www.seoulfn.com/news/articleView.html?i...,http://www.seoulfn.com/news/articleView.html?i...,2022-03-30 10:53:00
355,356,"카카오페이증권, 해외주식 소수점거래 시작",카카오페이증권이 해외 주식 소수점 거래 서비스를 시작했다. [사진: 카카오페이증권]...,http://www.digitaltoday.co.kr/news/articleView...,http://www.digitaltoday.co.kr/news/articleView...,2022-03-30 10:52:00
356,357,어렵다? 쉽다! 마음의 벽 확 낮춰줄 4色 '금융가 캐릭터' 열전,"# 카카오페이증권, 수익률에 따라 카카오프렌즈 테마 변화 사진=카카오페이증권 정식 ...",http://www.slist.kr/news/articleView.html?idxn...,http://www.slist.kr/news/articleView.html?idxn...,2022-03-30 10:44:00
357,358,"카카오페이증권, 해외 주식 소수점 거래 서비스 시작",카카오페이증권이 1000원부터 투자할 수 있는 해외 주식 소수점 거래 서비스를 28...,https://biz.sbs.co.kr/article_hub/20000056375?...,https://biz.sbs.co.kr/article_hub/20000056375?...,2022-03-30 10:39:00
358,359,"카카오페이증권, 해외 주식 소수점 거래 서비스 개시",카카오페이증권이 1000원부터 투자할 수 있는 해외 주식 소수점 거래 서비스를 지난...,https://www.khan.co.kr/economy/finance/article...,https://www.khan.co.kr/economy/finance/article...,2022-03-30 10:37:00
359,360,금감원 수시검사 첫 타깃 '토스'… 빅테크 검사 강화 차원,네이버파이낸셜은 지난해 12월 마이데이터 서비스 첫 날 개인정보 유출 사고가 일어났...,http://www.seoulwire.com/news/articleView.html...,http://www.seoulwire.com/news/articleView.html...,2022-03-30 10:36:00


In [ ]:
df_json['description']

0      20대의 차량은 토스, 토스뱅크, 토스증권, 토스페이먼츠 등 각 계열사로 골고루 전...
1      이어 청주페이 발행 총량 및 할인율 확대, 코로나 블루 극복 전담TF 구성 등을 공...
2      한 주주가 카카오·카카오페이 대표의 최저임금 선언을 언급하며 “기우성 대표와 서진석...
3      카카오페이로 10만 원 이상 결제할 경우 3000원 할인 쿠폰도 주어진다. 또한 티...
4      4위는 ‘네이버페이 우리카드 체크’다. 연회비 5000원으로 전월실적 조건 없이 네...
                             ...                        
995    한국은행이 23일 공개한 '전자지급 서비스 이용 현황'에 따르면 지난해 카카오페이 ...
996    비대면 거래 확대로 전자지급결제대행 서비스 이용액도 28%↑ 선볼전자지급·전자지급결...
997    카카오페이, 토스 등에서 미리 선불금을 충전해 물건을 사거나 교통요금을 납부하는 선...
998    삼성페이·카카오페이 등으로 대변되는 간편결제 이용금액도 6000억원을 돌파했다. 이...
999    선불전자지급서비스는 카카오페이, 토스 등에 선불금을 미리 충전해 교통요금, 상거래 ...
Name: description, Length: 1000, dtype: object

In [ ]:
description_list = df_json['description'].values.tolist()
description_slice = []
for i in range(0, len(description_list)):
    description_slice += description_list[i].split(' ')
len(description_slice)

26169

In [ ]:
title_list = df_json['title'].values.tolist()
title_slice = []
for i in range(0, len(title_list)):
    title_slice += title_list[i].split(' ')
len(title_slice)

6992

In [ ]:
desCount = {} 
 
for word in description_slice:
    desCount[word] = desCount.get(word, 0) + 1
    desKeys = sorted(desCount.keys())

In [ ]:
titleCount = {} 
 
for title in title_slice:
    titleCount[title] = titleCount.get(title, 0) + 1
    titleKeys = sorted(titleCount.keys())

In [ ]:
for word in desKeys:
    if desCount[word] > 50:
        print(word + ':' + str(desCount[word]))

:947
24일:52
28일:104
30일:51
거래:78
결제:162
내정자가:95
내정자는:62
대표:263
대표는:78
등:272
때까지:104
및:154
받겠다고:70
밝혔다.:173
서비스:65
서비스를:176
세븐카페:66
소수점:120
수:145
신:68
신규:74
신원근:338
신임:118
실시간:54
없이:58
오프라인:55
위한:129
있는:112
있다.:125
주가:52
주가가:106
주식:157
지난:88
지난해:110
최저임금만:85
추가:70
카카오:71
카카오페이:514
카카오페이,:87
카카오페이가:85
카카오페이는:159
통해:124
투자할:61
한:82
해외:101


In [ ]:
for word in titleKeys:
    if titleCount[word] > 20:
        print(word + ':' + str(titleCount[word]))

20만원:60
거래:48
결제:37
내정자:53
대표:140
대표,:21
대표이사:36
될:26
등:23
때까지:32
받겠다:48
서비스:39
선임:35
셀트리온:32
소수점:48
시작:38
신원근:171
신임:64
주가:86
주식:38
최저임금:48
최저임금만:44
카카오페이:174
카카오페이,:103
카카오페이증권,:50
해외:35
해외주식:22
